In [1]:
import pandas as pd
import plotly.express as px
import h3

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
simulation = pd.read_csv('simulation.csv')
print(simulation.shape)
simulation.head()

(7532, 9)


,drifting,steps_in_state,h3_index,month,mean_heading,ptt,date,lat,lon
0,True,1,8422d0bffffffff,12,0.0,129843,2013-12-19,53.980980,-166.800355
1,True,2,8422d0bffffffff,12,0.0,129843,2013-12-20,53.980980,-166.800355
2,True,3,8422d0bffffffff,12,0.0,129843,2013-12-21,53.980980,-166.800355
3,True,4,8422d43ffffffff,12,0.0,129843,2013-12-22,53.344588,-166.335720
4,True,5,8422d4dffffffff,12,0.0,129843,2013-12-23,52.829626,-165.313071


In [3]:
state = pd.read_csv('training_states.csv')
state['lat'] = state['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
state['lon'] = state['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])
print(state.shape)
state.head()

(7421, 9)


,ptt,h3_index,month,mean_heading,drifting,steps_in_state,date,lat,lon
0,129843,8422d0bffffffff,12,0.0,True,1.0,2013-12-19,53.980980,-166.800355
1,129843,8422d03ffffffff,12,0.0,True,2.0,2013-12-20,54.360925,-166.742418
2,129843,8422d03ffffffff,12,0.0,True,3.0,2013-12-21,54.360925,-166.742418
3,129843,8422d03ffffffff,12,0.0,True,4.0,2013-12-22,54.360925,-166.742418
4,129843,8422d03ffffffff,12,0.0,True,5.0,2013-12-23,54.360925,-166.742418


In [4]:
def plot_it(data, lat, lon, color):
    fig = px.scatter_geo(
        data, lat=lat, lon=lon, color=color,
    )
    fig.update_layout(autosize=True, height=600, geo=dict(center=dict(lat=58, lon=-150), projection_scale=6))
    return fig

In [5]:
plot_it(state, 'lat', 'lon', 'ptt')

In [6]:
plot_it(simulation, 'lat', 'lon', 'ptt')

In [7]:
plot_it(simulation[~simulation["drifting"]], 'lat', 'lon', 'ptt')

In [9]:
plot_it(simulation[simulation["drifting"]], 'lat', 'lon', 'ptt')